In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

# Load Dataset

In [2]:
data = pd.read_csv('applications data.csv')
data.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   record       1000000 non-null  int64 
 1   date         1000000 non-null  int64 
 2   ssn          1000000 non-null  int64 
 3   firstname    1000000 non-null  object
 4   lastname     1000000 non-null  object
 5   address      1000000 non-null  object
 6   zip5         1000000 non-null  int64 
 7   dob          1000000 non-null  int64 
 8   homephone    1000000 non-null  int64 
 9   fraud_label  1000000 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 76.3+ MB


In [4]:
data.shape

(1000000, 10)

In [5]:
data.isnull().sum()

record         0
date           0
ssn            0
firstname      0
lastname       0
address        0
zip5           0
dob            0
homephone      0
fraud_label    0
dtype: int64

# Fix Frivoulous Data

Replace the frivolous value with a field (the negative of the record number) that won't link 

In [6]:
# fix data type
## change date field from int to datetime 
data['date'] = pd.to_datetime(data['date'].astype(str))
## fill the left most blank with 0 if the number of digits < 5
data['zip5'] = data['zip5'].apply(lambda x: '{0:0>5}'.format(x))

# fix frivolous values
data.loc[data['ssn']==999999999,'ssn'] = -data.loc[data['ssn']==999999999,'record']
data['ssn'] = data['ssn'].apply(lambda x: '{0:0>9}'.format(x))

data.loc[data['dob']==19070626,'dob'] = -data.loc[data['dob']==19070626,'record']
data['dob'] = data['dob'].apply(lambda x:'{0:0>8}'.format(x))

data.loc[data['homephone']==9999999999,'homephone'] = -data.loc[data['homephone']==9999999999,'record']
data['homephone'] = data['homephone'].apply(lambda x:'{0:0>10}'.format(x))

#data.loc[data['address']=='123 MAIN ST','address'] = -data.loc[data['address']=='123 MAIN ST','record']
data.loc[data['address']=='123 MAIN ST','address'] = data.loc[data.address=='123 MAIN ST','record'].apply(lambda x: str(x)+' Record')

In [7]:
data

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,0216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,0132144161,0
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0
...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,766104848,XUSAAURMZ,EJXXEXZX,8079 EJRUJ AVE,54890,19550418,8236558223,0
999996,999997,2016-12-31,445178058,SAAJJEMTX,UMAUETJX,4817 UMSXT AVE,74683,19150624,6790134432,0
999997,999998,2016-12-31,025283410,ESRXXRSUX,EUEJRJJE,7562 RJMAZ ST,86805,19880628,3997657622,0
999998,999999,2016-12-31,361232148,XXMZSRXMA,RSRSUJSS,3408 UZZAM WY,49183,20120302,5935293731,0


# Feature Creation

### Create entities

In [8]:
data['name'] = data['firstname']+data['lastname']
data['name_dob'] = data['name']+data['dob']
data['fulladdress'] = data['address']+data['zip5']

### Create combinations of fields

In [9]:
data.columns

Index(['record', 'date', 'ssn', 'firstname', 'lastname', 'address', 'zip5',
       'dob', 'homephone', 'fraud_label', 'name', 'name_dob', 'fulladdress'],
      dtype='object')

In [11]:
%%time
combinations = ['ssn', 'firstname', 'lastname', 'address', 'zip5', 
                'dob', 'homephone', 'name_dob', 'fulladdress' ]

for i in range(len(combinations)):
    for j in range(i+1,len(combinations)):
        data[combinations[i]+'_'+combinations[j]]=data[combinations[i]]+data[combinations[j]]

CPU times: user 2.83 s, sys: 821 ms, total: 3.65 s
Wall time: 3.69 s


In [12]:
data.columns

Index(['record', 'date', 'ssn', 'firstname', 'lastname', 'address', 'zip5',
       'dob', 'homephone', 'fraud_label', 'name', 'name_dob', 'fulladdress',
       'ssn_firstname', 'ssn_lastname', 'ssn_address', 'ssn_zip5', 'ssn_dob',
       'ssn_homephone', 'ssn_name_dob', 'ssn_fulladdress',
       'firstname_lastname', 'firstname_address', 'firstname_zip5',
       'firstname_dob', 'firstname_homephone', 'firstname_name_dob',
       'firstname_fulladdress', 'lastname_address', 'lastname_zip5',
       'lastname_dob', 'lastname_homephone', 'lastname_name_dob',
       'lastname_fulladdress', 'address_zip5', 'address_dob',
       'address_homephone', 'address_name_dob', 'address_fulladdress',
       'zip5_dob', 'zip5_homephone', 'zip5_name_dob', 'zip5_fulladdress',
       'dob_homephone', 'dob_name_dob', 'dob_fulladdress',
       'homephone_name_dob', 'homephone_fulladdress', 'name_dob_fulladdress'],
      dtype='object')

In [14]:
# drop duplicate features
data.drop(columns = ['address_zip5','firstname_name_dob','lastname_name_dob',
                     'address_fulladdress','zip5_fulladdress','dob_name_dob',
                     'firstname_lastname'
                    ],inplace=True)

In [15]:
data.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,...,address_homephone,address_name_dob,zip5_dob,zip5_homephone,zip5_name_dob,dob_homephone,dob_fulladdress,homephone_name_dob,homephone_fulladdress,name_dob_fulladdress
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0,...,6861 EUTST PL1797504115,6861 EUTST PLXRRAMMTRSMJETJMJ000000-1,02765000000-1,027651797504115,02765XRRAMMTRSMJETJMJ000000-1,000000-11797504115,000000-16861 EUTST PL02765,1797504115XRRAMMTRSMJETJMJ000000-1,17975041156861 EUTST PL02765,XRRAMMTRSMJETJMJ000000-16861 EUTST PL02765
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,...,7280 URASA PL4164239415,7280 URASA PLMAMSTUJRRTTEMRRR19340615,5716919340615,571694164239415,57169MAMSTUJRRTTEMRRR19340615,193406154164239415,193406157280 URASA PL57169,4164239415MAMSTUJRRTTEMRRR19340615,41642394157280 URASA PL57169,MAMSTUJRRTTEMRRR193406157280 URASA PL57169
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,0216537580,0,...,5581 RSREX LN0216537580,5581 RSREX LNSZMMUJEZSEUSEZRAE000000-3,56721000000-3,567210216537580,56721SZMMUJEZSEUSEZRAE000000-3,000000-30216537580,000000-35581 RSREX LN56721,0216537580SZMMUJEZSEUSEZRAE000000-3,02165375805581 RSREX LN56721,SZMMUJEZSEUSEZRAE000000-35581 RSREX LN56721
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,0132144161,0,...,1387 UJZXJ RD0132144161,1387 UJZXJ RDSJJZSXRSZETJXTXXS19440430,3528619440430,352860132144161,35286SJJZSXRSZETJXTXXS19440430,194404300132144161,194404301387 UJZXJ RD35286,0132144161SJJZSXRSZETJXTXXS19440430,01321441611387 UJZXJ RD35286,SJJZSXRSZETJXTXXS194404301387 UJZXJ RD35286
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0,...,279 EAASA WY6101082272,279 EAASA WYSSSXUEJMSSSUUJXUZ19980315,0317319980315,031736101082272,03173SSSXUEJMSSSUUJXUZ19980315,199803156101082272,19980315279 EAASA WY03173,6101082272SSSXUEJMSSSUUJXUZ19980315,6101082272279 EAASA WY03173,SSSXUEJMSSSUUJXUZ19980315279 EAASA WY03173


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 42 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   record                 1000000 non-null  int64         
 1   date                   1000000 non-null  datetime64[ns]
 2   ssn                    1000000 non-null  object        
 3   firstname              1000000 non-null  object        
 4   lastname               1000000 non-null  object        
 5   address                1000000 non-null  object        
 6   zip5                   1000000 non-null  object        
 7   dob                    1000000 non-null  object        
 8   homephone              1000000 non-null  object        
 9   fraud_label            1000000 non-null  int64         
 10  name                   1000000 non-null  object        
 11  name_dob               1000000 non-null  object        
 12  fulladdress            100000

In [17]:
data.columns

Index(['record', 'date', 'ssn', 'firstname', 'lastname', 'address', 'zip5',
       'dob', 'homephone', 'fraud_label', 'name', 'name_dob', 'fulladdress',
       'ssn_firstname', 'ssn_lastname', 'ssn_address', 'ssn_zip5', 'ssn_dob',
       'ssn_homephone', 'ssn_name_dob', 'ssn_fulladdress', 'firstname_address',
       'firstname_zip5', 'firstname_dob', 'firstname_homephone',
       'firstname_fulladdress', 'lastname_address', 'lastname_zip5',
       'lastname_dob', 'lastname_homephone', 'lastname_fulladdress',
       'address_dob', 'address_homephone', 'address_name_dob', 'zip5_dob',
       'zip5_homephone', 'zip5_name_dob', 'dob_homephone', 'dob_fulladdress',
       'homephone_name_dob', 'homephone_fulladdress', 'name_dob_fulladdress'],
      dtype='object')

In [21]:
attributes = list(data.columns)
attributes.remove('record')
attributes.remove('date')
attributes.remove('firstname')
attributes.remove('lastname')
attributes.remove('zip5')
attributes.remove('fraud_label')

In [22]:
len(attributes)

36

In [24]:
attributes

['ssn',
 'address',
 'dob',
 'homephone',
 'name',
 'name_dob',
 'fulladdress',
 'ssn_firstname',
 'ssn_lastname',
 'ssn_address',
 'ssn_zip5',
 'ssn_dob',
 'ssn_homephone',
 'ssn_name_dob',
 'ssn_fulladdress',
 'firstname_address',
 'firstname_zip5',
 'firstname_dob',
 'firstname_homephone',
 'firstname_fulladdress',
 'lastname_address',
 'lastname_zip5',
 'lastname_dob',
 'lastname_homephone',
 'lastname_fulladdress',
 'address_dob',
 'address_homephone',
 'address_name_dob',
 'zip5_dob',
 'zip5_homephone',
 'zip5_name_dob',
 'dob_homephone',
 'dob_fulladdress',
 'homephone_name_dob',
 'homephone_fulladdress',
 'name_dob_fulladdress']

### Create relevant variables

In [25]:
%%time
finalDF = data[['record','date']]

for val in attributes:
    df1 = data[['record','date',val]]
    df2 = df1.copy()
    lags = [0,1,3,7,14,30]

    for lag in lags:
        temp_name = 'date_{}'.format(lag)
        df2[temp_name]=df2['date']+dt.timedelta(lag)

    dff = df1.merge(df2, on=val)

    # create variable: number of days since last seen
    tmp = dff[(dff['record_x']>dff['record_y'])][['record_x','date_y']].\
        groupby('record_x').last()['date_y'].reset_index()
    tmp.columns = ['record','last_seen']

    df1 = df1.merge(tmp, how = 'left', on = 'record')
    df1['#days_since'] = (df1['date'] - df1['last_seen']).dt.days
    df1['#days_since_start'] = (df1['date']-df1['date'].min()).dt.days
    df1['#days_since'] = df1['#days_since'].fillna(df1['#days_since_start'])
    df1.drop(columns = ['#days_since_start','last_seen'],inplace=True)

    # create variable: number of records with the same value over the last {0,1,3,7,14,30} days
    for lag in lags:
        temp = dff[(dff['record_x']>=dff['record_y']) & (dff['date_x']<=dff['date_{}'.format(lag)])]
        tempCount = temp[['record_x','record_y']].groupby('record_x').count().reset_index()
        temp_name = 'lag{}_count'.format(lag)
        tempCount.columns = ['record',temp_name]
        df1 = df1.merge(tempCount, on = 'record')
        
    for i in [1]:
        for j in [3, 7, 14, 30]:
            temp_name = 'lag{}_lag{}_avg'.format(i,j)
            numerator = 'lag{}_count'.format(i)
            denominator = 'lag{}_count'.format(j)
            df1[temp_name] = df1[numerator]/(df1[denominator]/j)
    
    df1.drop(columns = ['date',val], inplace = True)
    df1 = df1.set_index('record')
    df1 = df1.add_prefix('{}_'.format(val))
    finalDF = finalDF.merge(df1, left_on = 'record', right_index=True)

CPU times: user 3min 18s, sys: 1min 24s, total: 4min 42s
Wall time: 4min 51s


In [26]:
finalDF.shape

(1000000, 398)

In [27]:
finalDF

,record,date,ssn_#days_since,ssn_lag0_count,ssn_lag1_count,ssn_lag3_count,ssn_lag7_count,ssn_lag14_count,ssn_lag30_count,ssn_lag1_lag3_avg,...,name_dob_fulladdress_lag0_count,name_dob_fulladdress_lag1_count,name_dob_fulladdress_lag3_count,name_dob_fulladdress_lag7_count,name_dob_fulladdress_lag14_count,name_dob_fulladdress_lag30_count,name_dob_fulladdress_lag1_lag3_avg,name_dob_fulladdress_lag1_lag7_avg,name_dob_fulladdress_lag1_lag14_avg,name_dob_fulladdress_lag1_lag30_avg
0,1,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
1,2,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
2,3,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
3,4,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
4,5,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
999996,999997,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
999997,999998,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0
999998,999999,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,3.0,7.0,14.0,30.0


In [28]:
f = finalDF.copy()

In [30]:
%%time
# Creat variable: cross entity/combination variables
attr = ['ssn','address','dob','homephone','name']
for val in attr:
    a = attr.copy()
    a.remove(val)
    for val2 in a:
        df3 = data[['record','date',val,val2]].copy()
        df4 = df3.copy()
        lags = [0,1,3,7,14,30]

        for lag in lags:
            temp_name = 'date_{}'.format(lag)
            df4[temp_name]=df4['date']+dt.timedelta(lag)

        dff2 = df3.merge(df4, on=val)
        
        for lag in lags:
            temp = dff2[(dff2['record_x']>=dff2['record_y']) & (dff2['date_x']<=dff2['date_{}'.format(lag)])]
            tempCount = temp[['record_x','{}_y'.format(val2)]].groupby('record_x')['{}_y'.format(val2)].\
                                                            nunique().reset_index()
            tempCount.columns = ['record','{}_day_cross_count'.format(lag)]
            df3 = df3.merge(tempCount, on = 'record')
        df3.drop(columns = ['date',val,val2], inplace=True)
        df3 = df3.set_index('record')
        df3 = df3.add_prefix('{}_{}_'.format(val,val2))
        finalDF = finalDF.merge(df3, left_on='record', right_index=True)

CPU times: user 5min 22s, sys: 3min 30s, total: 8min 52s
Wall time: 9min 44s


In [31]:
finalDF

,record,date,ssn_#days_since,ssn_lag0_count,ssn_lag1_count,ssn_lag3_count,ssn_lag7_count,ssn_lag14_count,ssn_lag30_count,ssn_lag1_lag3_avg,...,name_dob_3_day_cross_count,name_dob_7_day_cross_count,name_dob_14_day_cross_count,name_dob_30_day_cross_count,name_homephone_0_day_cross_count,name_homephone_1_day_cross_count,name_homephone_3_day_cross_count,name_homephone_7_day_cross_count,name_homephone_14_day_cross_count,name_homephone_30_day_cross_count
0,1,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1
1,2,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1
2,3,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1
3,4,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1
4,5,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,2,1,1,1,1,1,2
999996,999997,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1
999997,999998,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1
999998,999999,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,1,1


In [32]:
f2 = finalDF.copy()

Create risk table variables

In [33]:
%%time
# split tt and oot (out-of-time data)
finalDF['dow'] = finalDF['date'].dt.dayofweek
finalDF = finalDF.merge(data[['record','fraud_label']], on='record')
train_test = finalDF[finalDF['date']<'2016-11-01']

In [39]:
finalDF = finalDF.merge(data[['record','fraud_label']], on='record')
train_test = finalDF[finalDF['date']<'2016-11-01']

In [40]:
finalDF

,record,date,ssn_#days_since,ssn_lag0_count,ssn_lag1_count,ssn_lag3_count,ssn_lag7_count,ssn_lag14_count,ssn_lag30_count,ssn_lag1_lag3_avg,...,name_dob_14_day_cross_count,name_dob_30_day_cross_count,name_homephone_0_day_cross_count,name_homephone_1_day_cross_count,name_homephone_3_day_cross_count,name_homephone_7_day_cross_count,name_homephone_14_day_cross_count,name_homephone_30_day_cross_count,dow,fraud_label
0,1,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,4,0
1,2,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,4,1
2,3,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,4,0
3,4,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,4,0
4,5,2016-01-01,0.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999996,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,2,1,1,1,1,1,2,5,0
999996,999997,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,5,0
999997,999998,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,5,0
999998,999999,2016-12-31,365.0,1,1,1,1,1,1,3.0,...,1,1,1,1,1,1,1,1,5,0


In [48]:
# do statistical smoothing
c = 4; nmid = 20; y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow')['fraud_label'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow-y_avg)/(1+np.exp(-(num-nmid)/c))
finalDF['dow_risk'] = finalDF['dow'].map(y_dow_smooth)

In [49]:
finalDF.shape

(1000000, 521)

# Output stats

In [50]:
finalDF.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
record,1000000.0,500000.500000,288675.278932,1.00000,250000.750000,500000.500000,750000.250000,1000000.000000
ssn_#days_since,1000000.0,163.703334,105.164178,0.00000,72.000000,155.000000,251.000000,365.000000
ssn_lag0_count,1000000.0,1.007305,0.223356,1.00000,1.000000,1.000000,1.000000,21.000000
ssn_lag1_count,1000000.0,1.014924,0.381210,1.00000,1.000000,1.000000,1.000000,34.000000
ssn_lag3_count,1000000.0,1.020140,0.423193,1.00000,1.000000,1.000000,1.000000,34.000000
...,...,...,...,...,...,...,...,...
name_homephone_14_day_cross_count,1000000.0,1.082454,0.590179,1.00000,1.000000,1.000000,1.000000,34.000000
name_homephone_30_day_cross_count,1000000.0,1.148827,0.815101,1.00000,1.000000,1.000000,1.000000,34.000000
dow,1000000.0,3.014974,1.994270,0.00000,1.000000,3.000000,5.000000,6.000000
fraud_label,1000000.0,0.014393,0.119104,0.00000,0.000000,0.000000,0.000000,1.000000


In [51]:
%%time
finalDF.describe().transpose().to_csv('variable_stats.csv') 

CPU times: user 11 s, sys: 2.67 s, total: 13.7 s
Wall time: 13.8 s


In [52]:
%%time
finalDF.to_csv('finalDF.csv') 

CPU times: user 2min 33s, sys: 4.32 s, total: 2min 37s
Wall time: 2min 38s
